#Who is this guy talking at me?

<img src="./gignosko_contact.png", align=left>

#iPython Notebook
iPython is a great substitute for the standard python interpreter. It gives a lot of additional features including tab completetion, line numbers, a host of magic functions that do things like time the execution of your code, data visualization support and a lot more. 

iPython also has this handy notebook, which is a great way to:
* Quickly edit and run code
* Take rich notes using markdown so you can pass them on to others
* Insert mathematical symbols using Latex
* Visualize data inline

#Libraries
The libraries we'll be using for this presentation are:
* pandas. This is a data analytics library that gives us an easy, familiar way to wrangle our data into shape along with access to powerful data analysis and graphing capabilities by tying in to the other libraries we'll look at. 
* matplotlib. The standard for plotting in python. This library has a wide variety of built in plot types and makes it relatively easy to get your data into a plot or multiple plots. 
* numpy. A powerful library for scientific computing in python. The core of numpy revolves around it's ndarray data structure, which allows for fast calculations for a wide range of mathematical uses.
* sklearn. Scikit Learn is a widely used Machine Learning package built on top of numpy, scipy and matplotlib. 

In [ ]:
import json
import pandas as pd

## Background on the data

Dataframe, a 2D structure of rows and columns, like a spreadsheet but...fun. 

Pandas Dataframes have a variety of semantics for accessing and slicing your data. For the most part, these resemble access semantics for common python data types such as dictionaries, lists and tuples, but they also have a bit of syntactic sugar to make access a little easier and a lot more powerful. 

Dataframe columns are actually a pandas Series datatype, which is a one dimensional data structure. We'll see them a bit in this, but for the most part we'll stick to the Dataframes.

In [41]:
# you will need to change these paths to fit your directory structure
current_leg_df = pd.read_csv('/Users/gignosko/congress/bills/legislators-current.csv')
pd.options.display.max_columns = 50

In [42]:
current_leg_df

,last_name,first_name,birthday,gender,type,state,party,url,address,phone,contact_form,rss_url,twitter,facebook,facebook_id,youtube,youtube_id,bioguide_id,thomas_id,opensecrets_id,lis_id,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,1952-11-09,M,sen,OH,Democrat,http://www.brown.senate.gov,713 Hart Senate Office Building Washington DC ...,202-224-2315,http://www.brown.senate.gov/contact,http://www.brown.senate.gov/rss/feeds/?type=al...,SenSherrodBrown,NaN,NaN,SherrodBrownOhio,UCgy8jfERh-t_ixkKKoCmglQ,B000944,136,N00003535,S307,5051,400050,27018,Sherrod Brown,gIQA3O2w9O,29389,Sherrod Brown
1,Cantwell,Maria,1958-10-13,F,sen,WA,Democrat,http://www.cantwell.senate.gov,311 Hart Senate Office Building Washington DC ...,202-224-3441,http://www.cantwell.senate.gov/public/index.cf...,http://www.cantwell.senate.gov/public/index.cf...,CantwellPress,NaN,NaN,SenatorCantwell,UCN52UDqKgvHRk39ncySrIMw,C000127,172,N00007836,S275,26137,300018,27122,Maria Cantwell,gIQAZxKkDP,39310,Maria Cantwell
2,Cardin,Benjamin,1943-10-05,M,sen,MD,Democrat,http://www.cardin.senate.gov,509 Hart Senate Office Building Washington DC ...,202-224-4524,http://www.cardin.senate.gov/contact/,http://www.cardin.senate.gov/rss/feeds/?type=all,SenatorCardin,senatorbencardin,1.204218e+14,senatorcardin,UCiQaJnMzlfzzG3VESgyZChA,C000141,174,N00001955,S308,4004,400064,26888,Ben Cardin,gIQAGMu99O,15408,Ben Cardin
3,Carper,Thomas,1947-01-23,M,sen,DE,Democrat,http://www.carper.senate.gov,513 Hart Senate Office Building Washington DC ...,202-224-2441,http://www.carper.senate.gov/public/index.cfm/...,http://www.carper.senate.gov/public/index.cfm/...,SenatorCarper,tomcarper,1.248911e+14,senatorcarper,UCgLnvbKwu4B3navofj6Qvvw,C000174,179,N00012508,S277,663,300019,22421,Tom Carper,gIQA3bm69O,15015,Tom Carper
4,Casey,Robert,1960-04-13,M,sen,PA,Democrat,http://www.casey.senate.gov,393 Russell Senate Office Building Washington ...,202-224-6324,http://www.casey.senate.gov/contact/,http://www.casey.senate.gov/rss/feeds/?all,SenBobCasey,SenatorBobCasey,1.077495e+14,SenatorBobCasey,UCtVssXhx-KuZa-hSvnsnJ0A,C001070,1828,N00027503,S309,47036,412246,2541,"Bob Casey, Jr.",gIQABeor9O,40703,"Bob Casey, Jr."
5,Corker,Bob,1952-08-24,M,sen,TN,Republican,http://www.corker.senate.gov,425 Dirksen Senate Office Building Washington ...,202-224-3344,http://www.corker.senate.gov/public/index.cfm?...,http://www.corker.senate.gov/public/index.cfm/...,SenBobCorker,bobcorker,1.092514e+14,senatorcorker,UCm7if_szz7DxfcdYGrywHSA,C001071,1825,N00027441,S310,1021114,412248,65905,Bob Corker,gIQADOuy9O,40705,Bob Corker
6,Feinstein,Dianne,1933-06-22,F,sen,CA,Democrat,http://www.feinstein.senate.gov,331 Hart Senate Office Building Washington DC ...,202-224-3841,https://www.feinstein.senate.gov/public/index....,http://www.feinstein.senate.gov/public/?a=rss....,SenFeinstein,senatorfeinstein,3.348873e+14,SenatorFeinstein,UCtVC--6LR0ff2aOP8THpuEw,F000062,1332,N00007364,S221,13061,300043,53273,Dianne Feinstein,gIQAnrCyDP,49300,Dianne Feinstein
7,Hatch,Orrin,1934-03-22,M,sen,UT,Republican,http://www.hatch.senate.gov,104 Hart Senate Office Building Washington DC ...,202-224-5251,http://www.hatch.senate.gov/public/index.cfm/c...,http://www.hatch.senate.gov/public/index.cfm/r...,SenOrrinHatch,NaN,NaN,SenatorOrrinHatch,UCQUPyPV_SR435OsPiFNIWPQ,H000338,1351,N00009869,S118,189,300052,53352,Orrin Hatch,gIQAzJiz9O,14503,Orrin Hatch
8,Klobuchar,Amy,1960-05-25,F,sen,MN,Democrat,http://www.klobuchar.senate.gov,302 Hart Senate Office Building Washington DC ...,202-224-3244,http://www.klobuchar.senate.gov/public/email-amy,NaN,NaN,NaN,NaN,senatorklobuchar,UCvdeJsDsV51tFb_hVqvtYGA,K000367,1826,N00027500,S311,83701,412242,65092,Amy Klobuchar,gIQA5G259O,40700,Amy Klobuchar
9,McCaskill,Claire,1953-07-24,F,sen,MO,Democrat,http://www.mccaskill.senate.gov,506 Hart Senate Office Building Washington DC ...,202-224-6154,http://www.mccaskill.senate.gov/?p=contact,http://mccaskill.senate.gov/rs

We can see that pandas has given us a neat data structure from the csv file. And in the next cells we can see how easy it is to slice a Dataframe and query for specific values. 

In [44]:
current_leg_df[:3]

,last_name,first_name,birthday,gender,type,state,party,url,address,phone,contact_form,rss_url,twitter,facebook,facebook_id,youtube,youtube_id,bioguide_id,thomas_id,opensecrets_id,lis_id,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,1952-11-09,M,sen,OH,Democrat,http://www.brown.senate.gov,713 Hart Senate Office Building Washington DC ...,202-224-2315,http://www.brown.senate.gov/contact,http://www.brown.senate.gov/rss/feeds/?type=al...,SenSherrodBrown,NaN,NaN,SherrodBrownOhio,UCgy8jfERh-t_ixkKKoCmglQ,B000944,136,N00003535,S307,5051,400050,27018,Sherrod Brown,gIQA3O2w9O,29389,Sherrod Brown
1,Cantwell,Maria,1958-10-13,F,sen,WA,Democrat,http://www.cantwell.senate.gov,311 Hart Senate Office Building Washington DC ...,202-224-3441,http://www.cantwell.senate.gov/public/index.cf...,http://www.cantwell.senate.gov/public/index.cf...,CantwellPress,NaN,NaN,SenatorCantwell,UCN52UDqKgvHRk39ncySrIMw,C000127,172,N00007836,S275,26137,300018,27122,Maria Cantwell,gIQAZxKkDP,39310,Maria Cantwell
2,Cardin,Benjamin,1943-10-05,M,sen,MD,Democrat,http://www.cardin.senate.gov,509 Hart Senate Office Building Washington DC ...,202-224-4524,http://www.cardin.senate.gov/contact/,http://www.cardin.senate.gov/rss/feeds/?type=all,SenatorCardin,senatorbencardin,1.204218e+14,senatorcardin,UCiQaJnMzlfzzG3VESgyZChA,C000141,174,N00001955,S308,4004,400064,26888,Ben Cardin,gIQAGMu99O,15408,Ben Cardin


This gives us a slice of the Dataframe, just like we might make of a list. 

In [45]:
current_leg_df[current_leg_df['last_name'] == 'Chu']

,last_name,first_name,birthday,gender,type,state,party,url,address,phone,contact_form,rss_url,twitter,facebook,facebook_id,youtube,youtube_id,bioguide_id,thomas_id,opensecrets_id,lis_id,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
82,Chu,Judy,1953-07-07,F,rep,CA,Democrat,http://chu.house.gov,2423 Rayburn HOB; Washington DC 20515-0527,202-225-5464,http://chu.house.gov/connect-with-me/email-judy,http://chu.house.gov/rss.xml,RepJudyChu,RepJudyChu,41228315130,RepJudyChu,UCfcbYOvdEXZNelM8T05nK-w,C001080,1970,N00030600,NaN,92573,412379,16539,NaN,gIQA1yiFAP,20955,Judy Chu


Here we take the Dataframe and tell pandas which element of the Dataframe we want by using a reference to the Dataframe itself. In this case we are asking it to find the last_name == 'Chu' as the slice Dataframe that we want. We'll see this method of slicing up the Dataframe a lot. 

In [47]:
val_s = current_leg_df['party'].value_counts()
val_s

Republican     301
Democrat       236
Independent      3
dtype: int64

Here we've grabbed the party column from the Dataframe and what pandas gives back is a Series, which is a one dimensional array. The value_counts() does what you'd expect, count all the values. And if we wanted to access those values...

In [49]:
val_s.values

array([301, 236,   3])

And the indices...

In [50]:
val_s.index

Index(['Republican', 'Democrat', 'Independent'], dtype='object')

And all together now...

In [53]:
print("There are", val_s.values[0], val_s.index[0], "in the list" )

There are 301 Republican in the list


Because this is a one dimensional array, the values are the counts and the party is the Series index. 

pandas can also read from a variety of file types, including json. But, there are some catches. Let's open one and see what we get.

In [54]:
loc = r'/Users/gignosko/congress/bills/votes/2013/h10/data.json'
single_vote_df = pd.read_json(loc)
single_vote_df

,category,chamber,congress,date,number,question,requires,result,result_text,session,source_url,type,updated_at,vote_id,votes
Aye,procedural,h,113,2013-01-15 00:18:00,10,On Motion to Adjourn,1/2,Failed,Failed,2013,http://clerk.house.gov/evs/2013/roll010.xml,On the Motion to Adjourn,2014-06-18 15:22:23,h10-113.2013,"[{'id': 'B000574', 'display_name': 'Blumenauer..."
No,procedural,h,113,2013-01-15 00:18:00,10,On Motion to Adjourn,1/2,Failed,Failed,2013,http://clerk.house.gov/evs/2013/roll010.xml,On the Motion to Adjourn,2014-06-18 15:22:23,h10-113.2013,"[{'id': 'A000055', 'display_name': 'Aderholt',..."
Not Voting,procedural,h,113,2013-01-15 00:18:00,10,On Motion to Adjourn,1/2,Failed,Failed,2013,http://clerk.house.gov/evs/2013/roll010.xml,On the Motion to Adjourn,2014-06-18 15:22:23,h10-113.2013,"[{'id': 'B001259', 'display_name': 'Braley (IA..."
Present,procedural,h,113,2013-01-15 00:18:00,10,On Motion to Adjourn,1/2,Failed,Failed,2013,http://clerk.house.gov/evs/2013/roll010.xml,On the Motion to Adjourn,2014-06-18 15:22:23,h10-113.2013,[]


Reading a json file into a Dataframe requires some type of orientation so pandas knows what are column headings versus indexes and how the values should be filled in. The default orientation for Dataframes is by column, so when pandas reads the json, the first level objects become the column headings for the dataframe. The second level objects, if there are any, become the row indexes. Everythong else becomes data.

Looking at the json file, there are 15 first level objects and 4 second level ones so, pandas has taken the first level of the json and turned it into columns and the turned the second level into rows. This isn't what we want, but it gives us a bit of a better insight into the data we have. 

The problem here is the shape of our json file. Because we have all the No votes clustered into a list and all the Aye votes clustered in a separate list, we'll need to dig in a little deeper into the json file and split it out. To do that we'll need to manually manipulate the file.

In [63]:
with open(loc) as json_file:
    json_data = json.load(json_file)  
    yes_df = pd.DataFrame(json_data['votes']['Aye'])
yes_df

,display_name,id,party,state
0,Blumenauer,B000574,D,OR
1,Conyers,C000714,D,MI
2,Hastings (WA),H000329,R,WA
3,Polis,P000598,D,CO


Here, we've read in the json file and passed only the Aye object in to the Dataframe. This gives us the Dataframe shape that we want, with the proper column headings and data. Since there were nested objects to use as the indeces, pandas defaulted to numbering from zero.

In [64]:
with open(loc) as json_file:
    json_data = json.load(json_file)  
    no_df = pd.DataFrame(json_data['votes']['No'])
no_df

,display_name,id,party,state
0,Aderholt,A000055,R,AL
1,Alexander,A000361,R,LA
2,Amash,A000367,R,MI
3,Amodei,A000369,R,NV
4,Andrews,A000210,D,NJ
5,Bachmann,B001256,R,MN
6,Bachus,B000013,R,AL
7,Barber,B001279,D,AZ
8,Barletta,B001269,R,PA
9,Barr,B001282,R,KY


We can do that for each of the 4 vote types, but that leaves us with 4 Dataframes, so we'll need a way to put them all into one workable Dataframe. Luckily, Dataframes have a concatenate function that'll do this for us. Since we won't have a way to tell where the Aye Dataframe ends and the No begins once we concat them, we'll add a column that records the vote for each row which is pretty easy to accomplish: we'll add a new column in the same way that we would add a new entry in a dictionary.

In [65]:
no_df['vote'] = 'No'
yes_df['vote'] = 'Aye'
yes_df

,display_name,id,party,state,vote
0,Blumenauer,B000574,D,OR,Aye
1,Conyers,C000714,D,MI,Aye
2,Hastings (WA),H000329,R,WA,Aye
3,Polis,P000598,D,CO,Aye


In [66]:
yes_no_df = pd.concat([no_df, yes_df])
yes_no_df

,display_name,id,party,state,vote
0,Aderholt,A000055,R,AL,No
1,Alexander,A000361,R,LA,No
2,Amash,A000367,R,MI,No
3,Amodei,A000369,R,NV,No
4,Andrews,A000210,D,NJ,No
5,Bachmann,B001256,R,MN,No
6,Bachus,B000013,R,AL,No
7,Barber,B001279,D,AZ,No
8,Barletta,B001269,R,PA,No
9,Barr,B001282,R,KY,No


In [ ]:
import os
home = r'/Users/gignosko/pytn/bills/'# you will need to change this for your directory path
votes = home + 'votes'
bills = home + 'bills'
votes_list = []
votes_df = pd.DataFrame()
counter = 0
for x in os.walk(votes):
    if x[2]:
        json_file = x[0] + '/' + x[2][0]
        json_data = open(json_file)
        all_data = json.load(json_data)
        
        if ('bill' in all_data) and ('amendment' not in all_data):
            bill_type = all_data['bill']['type']
            bill_number = bill_type + str(all_data['bill']['number'])
            bill_file = bills + '/' + bill_type + '/' + bill_number + '/data.json'
            with open(bill_file) as f:
                bill_json = json.load(f)
                sponsor_id = bill_json['sponsor']['thomas_id']
            sponsor_party = current_leg_df[current_leg_df['thomas_id'] == int(sponsor_id)]['party']
            if sponsor_party.values.size == 0:
                sponsor_party = historic_leg_df[historic_leg_df['thomas_id'] == int(sponsor_id)]['party']
            votes = all_data['votes']
            vote_id = all_data['vote_id']
            for k,v in votes.iteritems():
                df = pd.DataFrame(v)
                if sponsor_party.values:
                    df['sponsor_party'] = sponsor_party.values[0][0]
                else:
                    df['sponsor_party'] = 'U'
                df['vote'] = k
                df['bill'] = bill_number
                df['vote_id'] = vote_id
                votes_list.append(df)
            votes_df = pd.concat(votes_list)
    counter = counter + 1
votes_df.to_pickle('/Users/gignosko/PyTN/dataframe.pkl')# change this path, too. 

In [67]:
pickle_file = ('/Users/gignosko/PyOhio/dataframe.pkl')
total_df = pd.read_pickle(pickle_file)
total_df

,bill,display_name,first_name,id,last_name,party,sponsor_party,state,vote,vote_id
0,hr1120,Andrews,NaN,A000210,NaN,D,R,NJ,Yea,h100-113.2013
1,hr1120,Barber,NaN,B001279,NaN,D,R,AZ,Yea,h100-113.2013
2,hr1120,Barrow (GA),NaN,B001252,NaN,D,R,GA,Yea,h100-113.2013
3,hr1120,Bass,NaN,B001270,NaN,D,R,CA,Yea,h100-113.2013
4,hr1120,Beatty,NaN,B001281,NaN,D,R,OH,Yea,h100-113.2013
5,hr1120,Becerra,NaN,B000287,NaN,D,R,CA,Yea,h100-113.2013
6,hr1120,Bera (CA),NaN,B001287,NaN,D,R,CA,Yea,h100-113.2013
7,hr1120,Bishop (GA),NaN,B000490,NaN,D,R,GA,Yea,h100-113.2013
8,hr1120,Bishop (NY),NaN,B001242,NaN,D,R,NY,Yea,h100-113.2013
9,hr1120,Blumenauer,NaN,B000574,NaN,D,R,OR,Yea,h100-113.2013


You'll notice here that the index resets because each of the Dataframes that we started with has it's own index and pandas doesn't automatically re-index on concatenation, but that's easy to fix if we need to.

In [68]:
total_df.index = range(1, len(total_df) +1)
total_df

,bill,display_name,first_name,id,last_name,party,sponsor_party,state,vote,vote_id
1,hr1120,Andrews,NaN,A000210,NaN,D,R,NJ,Yea,h100-113.2013
2,hr1120,Barber,NaN,B001279,NaN,D,R,AZ,Yea,h100-113.2013
3,hr1120,Barrow (GA),NaN,B001252,NaN,D,R,GA,Yea,h100-113.2013
4,hr1120,Bass,NaN,B001270,NaN,D,R,CA,Yea,h100-113.2013
5,hr1120,Beatty,NaN,B001281,NaN,D,R,OH,Yea,h100-113.2013
6,hr1120,Becerra,NaN,B000287,NaN,D,R,CA,Yea,h100-113.2013
7,hr1120,Bera (CA),NaN,B001287,NaN,D,R,CA,Yea,h100-113.2013
8,hr1120,Bishop (GA),NaN,B000490,NaN,D,R,GA,Yea,h100-113.2013
9,hr1120,Bishop (NY),NaN,B001242,NaN,D,R,NY,Yea,h100-113.2013
10,hr1120,Blumenauer,NaN,B000574,NaN,D,R,OR,Yea,h100-113.2013


Now that we've started munging our data into something a bit more useful, let's start looking at it. 

pandas lets you group your Dataframes by columns, much like you could in a pivot table. We can use groupby() to group our columns and then use count() to get some counts of the values. Let's play around with this a bit and see how we can drill in to get a better idea of the data.

In [70]:
total_df.groupby(['party'])['vote'].count()

party
D    147398
I       292
R    168672
Name: vote, dtype: int64

You can see here that we've grouped by the party column, but we've told pandas to give us the count() of the vote column.  The outcome is a Series with party as the index and the counts as the values. The groupby() function always returns a GroupBy object, which has a variety of functions you can call to return a  Series with the grouping as the index.

We can group by more than one column, also.

In [71]:
total_df.groupby(['bill', 'vote_id','party'])['vote'].count()

bill        vote_id        party
hconres105  h452-113.2014  D        199
                           R        233
hconres15   h43-113.2013   D        200
                           R        231
hconres18   h77-113.2013   D        200
                           R        231
hconres19   h78-113.2013   D        200
                           R        231
hconres25   h88-113.2013   D        200
                           R        232
hconres44   h376-113.2013  D        200
                           R        233
hconres58   h526-113.2013  D        200
                           R        231
hconres96   h177-113.2014  D        199
                           R        233
hjres106    s11-113.2014   D         53
                           I          2
                           R         45
hjres124    h508-113.2014  D        199
                           R        232
            h509-113.2014  D        199
                           R        232
            s269-113.2014  D         53
       

In this one, we've dug a little deeper into the groupings so we can see how many total votes there were by party for each bill, vote_id and party. Since this is a Series everything other than the count is the index and we can see more about that a little further below. 

In [72]:
vote_counts = total_df.groupby(['bill', 'vote_id', 'sponsor_party','party', 'vote'])['vote'].count()
vote_counts

bill        vote_id        sponsor_party  party  vote      
hconres105  h452-113.2014  D              D      Nay             3
                                                 Not Voting      6
                                                 Yea           190
                                          R      Nay            37
                                                 Not Voting     16
                                                 Yea           180
hconres15   h43-113.2013   R              D      Nay           186
                                                 Not Voting     14
                                          R      Nay             4
                                                 Not Voting      5
                                                 Yea           222
hconres18   h77-113.2013   R              D      Not Voting     31
                                                 Yea           169
                                          R      Not Voting     12
  

In [74]:
vote_counts.index[1]

('hconres105', 'h452-113.2014', 'D', 'D', 'Not Voting')

Here we can see that the index of this Series is actually a tuple.

So, let's back up just a bit and build a new DataFrame to work with.


First, we create a new Dataframe to work with because we only need a few of the columns from the total_df. To get specific columns, we can use the Dataframe.loc function, which grabs the rows or columns by label name. So, total_df.loc[:,...] is a slice that says grab me all the rows while ..., ['bill', 'vote_id','party', 'vote']] says grab me those particular labels. This makes our new Dataframe.

In [75]:
new_df = total_df.loc[:, ['bill', 'vote_id','party', 'vote']]
new_df

,bill,vote_id,party,vote
1,hr1120,h100-113.2013,D,Yea
2,hr1120,h100-113.2013,D,Yea
3,hr1120,h100-113.2013,D,Yea
4,hr1120,h100-113.2013,D,Yea
5,hr1120,h100-113.2013,D,Yea
6,hr1120,h100-113.2013,D,Yea
7,hr1120,h100-113.2013,D,Yea
8,hr1120,h100-113.2013,D,Yea
9,hr1120,h100-113.2013,D,Yea
10,hr1120,h100-113.2013,D,Yea


In [77]:
temp_s1 = new_df.groupby(['vote_id','bill', 'party','vote'])['vote'].count()
temp_s1

vote_id        bill      party  vote      
h10-113.2014   hr2279    D      Aye             5
                                No            184
                                Not Voting     11
                         R      Aye           220
                                No              4
                                Not Voting      8
h100-113.2013  hr1120    D      Nay             1
                                Not Voting      3
                                Yea           197
                         R      Nay           228
                                Not Voting      3
h100-113.2014  hres501   D      Nay           190
                                Not Voting      8
                                Yea             1
                         R      Not Voting      7
                                Yea           224
h101-113.2013  hr1120    D      No            199
                                Not Voting      2
                         R      Aye           219
       

We have the count values that we want now in that Series, but we really want to get everything flipped around so those index values get pushed up to be column headings. We can do that with the unstack() function, which takes a Series and returns a Dataframe with the Series indeces as column headings. 

In [ ]:
temp_df2 = temp_s1.unstack()
temp_df2

Next, we create a series from that Dataframe, the same as we did above, so that the vote_id, bill, party and vote all make up a  unique index with the counts as the data values. Then, we unstack the Series index, which pulls the index apart and turns each hierarchical level into a column again, but this time, it takes the counts values with it and puts them in as data. 

But, we have a lot of NaN values in this Dataframe and that's going to mess us up later so let's change those to 0. pandas has a function fillna() that works on both Datframes and Series to fill in the NaN values with whatever value you pass as a function parameter. By default the fillna() function returns a copy of the Dataframe as a new object, but we can tell pandas that we want to fill the values on the existing Dataframe by passing the inplace parameter as True.

In [ ]:
temp_df2.fillna(0, inplace=True)
temp_df2

Ok, let's start visualizing!

In [ ]:
%matplotlib inline
temp_df2[:10].plot(kind='bar', stacked=True)

matplotlib is the most widely used plotting library in python and the standard tool for plotting when working with pandas because it's integrated into the pandas library. Above, you can see that we've called the plot() function on a slice of our Dataframe, passed in the kind of plot we want and since it's a bar, told it to stack the values from each column. You can see that we've let pandas decide where to put the legend and it's obscuring out data. If we import matplotlib and manually construct this plot, then we can gain a bit more control over things. We'll do some of that below. 

Also notice the %matplotlib inline.  This tells iPython notebook to not only return the plot, but to also show it inline. Without that line, the function would return the plot object, but we wouldn't see the pretty pictures. 

As we're looking at this plot, something obvious stands out. We have a column for Yea and a column for Aye; likewise, we have a column for No and a column for Nay. Welcome to the Federal Government! Each House of Congress records it's votes differently, so we get two columns for the same answer. Let's fix that. 

In [ ]:
temp_df2['Y'] = temp_df2['Aye'].add(temp_df2['Yea'])
temp_df2['N'] = temp_df2['No'].add(temp_df2['Nay'])
temp_df2 = temp_df2.drop(['Aye', 'Yea', 'Nay', 'No', 'Not Voting', 'Present'], axis=1)
temp_df2

First, we created a new column labeled "Y" and took the values of the Aye column and added to them the values of the Yea column. We then did the same for the "N" column to collect all the no votes and then we dropped all the columns we weren't interested in. Dropping requires an axis so pandas knows if you want to drop rows or columns, so we've passed in the axis=1 parameter. 

In [ ]:
temp_df2[:10].plot(kind='barh', stacked=True)

This looks better. Pandas will only try to plot columns with numeric values and now we only have two of those. We've also turned the plot into a horizontal bar plot, which makes the y-axis labels easier to read. 

Below, we take a look at how to manually create plots by calling matplotlib directly.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
temp_df2['sum'] = temp_df2.sum(axis=1)
x = temp_df2['sum'].values 
y = np.square(x)
plt.ylabel('No. votes')
plt.xlabel('Sum of all votes')
plt.scatter(x=x, y=y)

Here we've added a sum column by calling the sum() function and telling it to work across the columns. We've also imported matplotlib so we'll have better control over the plots and numpy so we can use some of it's math functions. 

We set the values of the sum column as the x axis and set y as a function of the squares. Notice we passed a Series as the x-axis values. pandas Series and Dataframes are both built on top of numpy's ndarray data type, so we've essentially passed an array as the x-axis. The y-axis is even more impressive: instead of passing in a collection of values, we've passed in a function and pandas will do the calculations for us! This way we get a nice curve in our scatterplot. We've also manually added labels. Although matplotlib took care of the tick marks on the x and y axes for us, we could have given ranges for those and adjusted the scale of the plot if we had wanted. 

In [ ]:
from scipy import stats
x = temp_df2['Y']
y = temp_df2['N']
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
yp = np.polyval([slope, intercept],x)
plt.plot(x, yp)
plt.scatter(x,y)

Using scipy's stats library we can call the linregress() function, which gives us back a tuple with everything we could want from a linear regression. We can then take the slope and intercept and pass them in to the numpy polyval() function, which essentially solves polynomial equations for specific values. Here, the polynomial is the well-known y = mx+b where m is the slope and b is the intercept. It gets solved for y with x being all the values of the 'Y' column from the DataFrame. We also did a scatter plot of the 'Y' and 'N' values and plotted both the scatter and the lnear regression in the same plot. 

This visualization shows a distinct split in votes, with a group of them in the lower left, and a much larger grouping of them above. This is because we've mixed House and Senate votes.  The House will have up to 435 votes per vote_id, while the Senate could have no more than 100.  This split in the groups also brings the line of the regression down quite a bit. 

Below we see an interesting grouping of plots called a scatter matrix, which does a scatter plot of every column over every other column. Since the diagonals in this matrix would be a straight line (plotting a column over itself), the scatter_matrix function takes a diagonal parameter that lets us tell it what we want on the diagonal. In this case we're plotting the kernal density estimate, which is roughly equivalent to a histogram, but with a smooth curve. 

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(temp_df2, alpha=0.2, figsize=(10,10), diagonal='')

#Machine Learning
In *A Brief History of Time* Stephen Hawking said that he was warned that for each equation he added, the readership would drop by half. I certainly hope that's not the case with this audience, but just in case, we'll just look at one equation.

One appplication of machine learning is to help us categorize new data and this categorization falls into roughly two type: supervised learning, where we know the classification of the training data before we train, and unsupervised learning, where we let the learning algorithm cluster the data and figure out classifications. We're going to look at a supervised method, which means that we will train the machine using input for which we already know the output. For input we'll use the party of the vote sponsor along with the party of the voter and the output is the vote each voter cast.  We'll be using a K-nearest neighbor classifier for our learning algorithm and we'll go over it in a bit more detail below.  

## K Nearest Neighbors
KNN can be used for either classification or for regression. We'll be using it to classify the votes from our data set, first giving the algorithm a portion of the data to train on and another portion to test and see how well we've trained it. As the name of the algorithm implies, KNN classifies new input based on it's "distance" from it's k closest neighbors. The distance formula most often used is the Minkoski formula, the one equation we'll look at (I promise). 

Minkoski distance

$$X = (x_1, x_2, ... x_n) Y = (y_1, y_2, ..., y_n)$$

$$D = {\left({\sum_{l=1}^n {\left|x_i - y_i\right|}^p}\right)}^{1/p}$$

THe X and Y here are essentially the same as the x and y of a Cartesian plane, but expanded out to multiple (technically infinite) dimensions with X being the set of all dimensions of the point to test and Y being the set of all dimensions of the neighboring point, for which we already know the classification. The formula takes the absolute value of the difference between the first dimension of the two points, raises that to p, then adds in the second dimension and so on through all the dimensions. It then raises that whole sum to 1/p, which is the same thing as taking the p-root. The default for p is 2, which makes this the eqivalent of the Euclidean distance, for all the math geeks out there that aren't afraid of an equation or two. 

K is the number of neighbors we want to survey to determine the classification for our new point; if K =1 then the class of the test sample is the same as the class of the nearest neighbor and for K > 1, the class of the test sample is most common class amoung those K neighbors.

One thing to watch out for with this classifier is that KNN is susceptible to noise because it muddles the distance formula. Noise is any dimension that is irrelevent to the decision and in this case the bill number, the vote number, the voter id, and any names are all noisy because they don't add to the prediction of future votes in our case. So, let's rework the data a bit more to get rid of the noise.

In training a classifier, we need the data that is used as the input and the data that is the actual known classification so we can give those separately to the classifier. We can feed the KNN classifier the data in an "array like" structure, which can be the Dataframe, but the target (or the known outputs, which in this case is the actual vote that was cast) needs to be an actual array, so we'll have to split that off. To get the data into the proper form for the classifier, we're going to go back to our total_df, but we need to drop some un-needed columns, clean out some data and change what's left to numeric data.

Luckily for us, panda's Dataframes and Series are both built on top of numpy's ndarray so getting the data in the right format is straightforward. An ndarray is different from the standard python array in that the ndarray is an n-dimensional array, rather than a 1 dimensional array and ndarrays give us access to a lot of numpy's mathematical functions. 

For now, what we need to do is pull the values of the target (the actual vote cast) out of the Dataframe and they will automatically be an ndarray, however the values will be be object datatypes, so we'll need to convert them to int because that's what the classifier needs. The data is represented as a float because it typically needs to be normalized to a value between 0 and 1; since we only have two values per input (Democrat or Republican), we can just go ahead and make them 0 and 1 for simplicity.  And, since the data can be in a Dataframe, we'll just change it in line.

Let's start by creating our final Dataframe and dropping the unnecessary columns which is an easy task in pandas. We can create a list of column names we'd like to drop, then use that list as a parameter to the drop() function; we will also need to tell pandas if we are dropping columns or rows, so we'll set the index parameter to 1 for columns. For a bit of simplicity, we want to make sure we only have Democrats and Republicans (Sorry, Senator Sanders) and Yes or No votes.  We can tell pandas to re-create the Dataframe with vote values not equal to Present or Not Voting and with party and sponsor_party without I's. 

In [ ]:
drop_list = ['bill', 'display_name', 'first_name', 'id','last_name', 'state', 'vote_id']
final_df = total_df.drop(drop_list, axis=1)
#final_df['vote'].value_counts()
final_df

In [ ]:
change_mapping = {'R': 0, 'D': 1, 'I': 2, 'Nay': 0, 'No': 0, 'Yea': 1 , 'Aye': 1, 'Yes': 1, 'Not Voting': 2, 'Present':3}
final_df['sponsor_party'] = final_df['sponsor_party'].map(change_mapping)
final_df['party'] = final_df['party'].map(change_mapping)
final_df['vote'] = final_df['vote'].map(change_mapping)
final_df

What we've just done is used Pandas map function to map a dictionary across all the values in the various Series of our Dataframe. Pandas takes the value in a cell, looks it up in the change_map dict as a key and then replaces it with the corresponding value. We could also have passed in a function rather than a dict and Pandas would use the value as the parameter to the function and the return value as the new cell value. 

In [ ]:
target = final_df.take([2], axis=1)
target = target.values
target = target.astype('int')
target = target.ravel()
target


Above, we've created the target Series by taking a Series out of our Dataframe, grabbed the values, turned them into ints. The what we would have back is an n dimensional array, or an array of arrays, where each value is in it's own array, all of which are inside an array, so we used the ravel() function to turn the multidimensional array into a single dimensional array.

In [ ]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4, weights='distance')
from sklearn import cross_validation as cv
data = final_df.drop('vote', axis=1)
data_train, data_test, target_train, target_test = cv.train_test_split(data, target, test_size=0.1)
#simple_knn = knn.fit(data_train, target_train)
simple_knn = pickle.load(open('/Users/gignosko/PyOhio/simple_knn.pkl', 'rb'))


In [ ]:
import pickle
pickle.dump(simple_knn, open('/Users/gignosko/PyOhio/simple_knn.pkl', 'wb'))

In [ ]:
pickle.dump(predicted, open('/Users/gignosko/PyOhio/simple_predicted.pkl', 'wb'))

In [ ]:
pickle.dump(simple_score, open('/Users/gignosko/PyOhio/simple_score.pkl', 'wb'))

In [ ]:
predicted = pickle.load(open('/Users/gignosko/PyOhio/simple_predicted.pkl', 'rb'))
#predicted = simple_knn.predict(data_test)
print( "Prediction " + str(predicted))
print( "Actual     " + str(target_test))


In [ ]:
#simple_score = simple_knn.score(data_test, target_test)
#pickle.dump(simple_score, open('/Users/gignosko/PyOhio/simple_score.pkl', 'wb'))
simple_score = pickle.load(open('/Users/gignosko/PyOhio/simple_score.pkl', 'rb'))
print( "Accuracy   " + str(simple_score*100) + "%")

In [ ]:
print(simple_knn.kneighbors(data_test[:1], 5))

In [ ]:
print(data_train.iloc[[290]])
print(data_test[:1])

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = predicted
cm = confusion_matrix(target_test, y_pred)
print(cm)
import matplotlib.pylab as plt
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
state_df = pd.read_csv('/Users/gignosko/PyOhio/state_table.csv')
state_df

In [ ]:
region_dict = {k: ''.join(v["census_region_name"].tolist()) for k,v in state_df.groupby("abbreviation")}
region_dict
final_df['region'] = total_df['state'].map(region_dict)
final_df


In [ ]:
region_mapping = {'South': 1, 'West': 2, 'Midwest': 3, 'Northeast': 4, }
final_df['region'] = final_df['region'].map(region_mapping)
final_df

In [ ]:
data = final_df.drop('vote', axis=1)
data_train, data_test, target_train, target_test = cv.train_test_split(data, target, test_size=0.1)
#noisy_knn = knn.fit(data_train, target_train)
#pickle.dump(noisy_knn, open('/Users/gignosko/PyOhio/noisy_knn.pkl', 'wb'))
noisy_knn = pickle.load(open('/Users/gignosko/PyOhio/noisy_knn.pkl', 'rb'))

In [ ]:
#noisy_predicted = noisy_knn.predict(data_test)
noisy_predicted = pickle.load(open('/Users/gignosko/PyOhio/noisy_knn.pkl', 'rb'))
print( "Prediction " + str(predicted))
print( "Actual     " + str(target_test))

In [ ]:
pickle.dump(predicted, open('/Users/gignosko/PyOhio/noisy_predicted.pkl', 'wb'))

In [ ]:
#noisy_score = noisy_knn.score(data_test, target_test)
#pickle.dump(noisy_score, open('/Users/gignosko/PyOhio/noisy_score.pkl', 'wb'))
noisy_score = pickle.load(open('/Users/gignosko/PyOhio/noisy_score.pkl', 'rb'))
print( "Accuracy   " + str(noisy_score*100) + "%")

State stuff below

Pandas Series have a map function that will take a function or a dict and apply it across all the rows in the Series. We can use this to create a mapping from the state abbreviation to the region for that state and add that as a new column in our total_df Dataframe. To get the dict for the mapping takes a little bit of magic, though. The 'key' for the dictionary comprehension is straighforward enough; when we groupby('abbreviation'), the state abbreviation becomes the index and gets returned as the key in the for...in syntax. The 'value' is a little harder. Since everything other than the abbreviation gets returned as the value, we need to dig into that and get just the 'census_region_name'. But, since this returns a Series, in this context we get a lot of unneeded information, such as the Name and the index. To strip this away we use the Series tolist(). The documentation for tolist() is a bit, um...terse. "Convert Series to a nested list". What it ends up doing here is taking the value and putting it into a list, so the documentation is terse, but spot on. To get it out of a list and into a string we use the pythonic ''.join([list]) form. We can then use the resulting dict of {state: region} and map it across the state column of the Dataframe to add a new column with the region. Whew!

We can see that we have the same problem as earlier, which is both Yea and Aye for yes and Nay or No for no. We fixed it before pretty easily, but now we need to make two changes: combine all the yes/no values and convert them from text to numbers. pandas lets us map a dictionary to a Series, so we can create a simple dictionary that spells out our changes and then map it to the individual Series in the Dataframe. 

The problem above is that we have no way to get back to the full original entry.

Also, 50% isn't great. We're missing a lot of useful data such as the content of the bill, i.e., is it a controversial bill (healthcare, gun control, etc) or a no brainer (naming a highway after a former president). 

The confusion matrix is a way of plotting the actual test outcomes versus the predicted outcomes. If you look at the labels, the True label is plotted on the y-axis and the Predicted label is on the x-axis. Looking along the diagonal we see in the top left the count of of where the classifier correctly classified the Predicted 0 to be the same as the True 0 and in the bottom right we see where the classifier correctly classified the Predicted 1 as the True 1. The other blocks show where the classifier miss classified (or confused) one class with the other. 

#Below is old!

In [ ]:
data = final_df.drop('vote', axis=1)

In [ ]:
from sklearn.linear_model import perceptron
from sklearn import cross_validation as cv
per = perceptron.Perceptron(n_iter=10, eta0=0.002)
data_train, data_test, target_train, target_test = cv.train_test_split(data, target)
per.fit(data_train, target_train)
print "Prediction " + str(per.predict(data_test))
print "Actual     " + str(target_test)
print "Accuracy   " + str(per.score(data_test, target_test)*100) + "%"

Here we've done several things. After our imports we instantiate a Perceptron with the number of iterations set to 10 and a learning rate (eta0) of 0.002; this can in theory help reduce the number of iterations the Perceptron needs. Often times we will have one set of data and we'll want to split it so we have a a training set and a testing set. sklearns's train_test_split function will do that for us, returning a pair of data and target arrays. The fit() function does the actual training of the Perceptron. It takes the a data parameter to run through the Perceptron algorithm and a target parameter which are the known values of the training set. Taken together, these train the Perceptron how to handle new data so it can be applied to data with unknown targets. 

Next, we run the predict() function on the test data. This takes the trained Perceptron (which is essentially just the best set of weights from the training iterations) and uses it to predict what the output from the test data should be. When we print out the Accuracy, we can see that we haven't done very well at all. We'd love to have an accuracy above 80% to feel confident about our Perceptron. 

Most likely, we just haven't set up a very good model. We've cut a lot of corners to get somethign going and that's probably tripped us up. First, we've put both the House and the Senate voting data together into one Dataframe, and that's going to skew things because the Senate could only record 100 votes per vote id, while the House could record up to 435. Second, we looked at all bills evenly, meaning we didn't take out procedural votes such as setting the agenda or non-divisive votes such as naming a highway after our favorite Doctor (10, right?) We really need a subject matter expert to help comb through the bills and guide us to the more inflamatory votes, but as we progress, maybe we can start to find better ways to classify this data. 

Let's look at one last way of evaluating the Perceptron that you'll likely run into through data analysis and machine learning: the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = per.predict(data_test)
cm = confusion_matrix(target_test, y_pred)
print cm
import matplotlib.pylab as plt
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

The confusion matrix is a way of plotting the actual test outcomes versus the predicted outcomes. If you look at the labels, the True label is plotted on the y-axis and the Predicted label is on the x-axis. Looking along the diagonal we see in the top left the count of of where the Perceptron correctly classified the Predicted 0 to be the same as the True 0 and in the bottom right we see where the Perceptron correctly classified the Predicted 1 as the True 1. The other blocks show where the Perceptron miss classified (or confused) one class with the other. 

There's a lot here and we've really just scratched the surface of the data analysis, machine learning and the python tools we can use for each. In the near future I hope to tackle a type of unsupervised learning: clustering. 

If you have any questions or comments, please feel free to reach out to me through comments, pull requests or on twitter: @\_gignosko\_

Thanks!